In [ ]:
import datetime
import requests
import json
import pandas as pd
import time

LOCALE = 'ko-KR'

# Band Key 찾기

#### 수정해야할 파라미터

In [ ]:
###################### 수정해야할 파라미터 ######################
ACCESS_TOKEN = 'YOUR_TOKEN'
HEADERS = {'X-Naver-Client-Id':'YOUR_ID', 'X-naver-Client-secret':'YOUR_PW'} # Your id and pw
BAND_NAME = 'BANDNAME What you want crawl'
NOW_VERSION = 2
###################### 수정해야할 파라미터 ######################

In [ ]:
# 내가 가입된 밴드명 확인 및 밴드 key 확인
url = f'https://openapi.band.us/v2.1/bands.json?access_token={ACCESS_TOKEN}'
r = requests.get(url)
r_json = json.loads(r.text)
band_df = pd.DataFrame(r_json['result_data']['bands'])
band_df

In [ ]:
# 밴드키 받아오기
BAND_KEY = band_df.loc[band_df['name'] == BAND_NAME, 'band_key'].values[0]
BAND_KEY

# 정보 크롤링 

### 처음 수집하는 것인지, 기존에 수집하던 것을 이어서 하는 것인지에 따라 셀을 다른 것을 선택합니다.
 - 만약 처음 수집한다면, 첫번째 셀을 실행, 두번째 셀은 실행하지 않습니다.
 - 만약 두번째로 수집한다면, 첫번째 셀은 실행하지 않고 두번째 셀만 실행합니다.

In [ ]:
################ 처음시작 시 ################
call_count = 1
NOW_VERSION = 1
API_FILE_SAVE_PATH = f'../output/{BAND_NAME}_API_ver{NOW_VERSION}(after값).xlsx'
DATAFRAME_SAVE_PATH = f'../output/{BAND_NAME}_API_ver{NOW_VERSION}.xlsx'

print(f'call_count: {call_count}')
################ 처음시작 시 ################

In [ ]:
################ 수동작동시 ####################### --> 처음 선택 시 해당 셀은 선택하지 않습니다!
API_FILE_PATH = f'../output/{BAND_NAME}_API_ver{NOW_VERSION}(after값).xlsx'
API_FILE_SAVE_PATH = f'../output/{BAND_NAME}_API_ver{NOW_VERSION+1}(after값).xlsx'
DATAFRAME_SAVE_PATH = f'../output/{BAND_NAME}_API_ver{NOW_VERSION+1}.xlsx'
after_df = pd.read_excel(API_FILE_PATH)

call_count = (after_df.shape[0]) * NOW_VERSION +1 # 처음시작 시 call_count는 1을 넣어라.
after = after_df['after값'].tolist()[-1]

print(f'call_count: {call_count}')
print(f'after: {after}')
################ 수동작동시 #######################

In [ ]:
df = pd.DataFrame()
t1 = time.time()
after_dict = {}

while True:
    
    # url 불러오기
    if call_count == 1:
        url = f'https://openapi.band.us/v2/band/posts.json?access_token={ACCESS_TOKEN}&band_key={BAND_KEY}&locale={LOCALE}'
        r = requests.get(url, headers=HEADERS, timeout = 10)
        r_json = json.loads(r.text)
        
    else:
        url = f'https://openapi.band.us/v2/band/posts.json?access_token={ACCESS_TOKEN}&band_key={BAND_KEY}&locale={LOCALE}&after={after}'
        r = requests.get(url, headers=HEADERS, timeout = 10)
        r_json = json.loads(r.text)
        
    # 사용횟수 초과 여부 확인
    if r_json['result_code'] == 1002:
        print('오늘 사용횟수를 초과하였습니다.')
        break
        
    # after 파라미터 정의 및 stopping조건 정의
    if r_json['result_data']['paging']['next_params'] == None:
        print('다음 after 파라미터가 존재하지 않습니다. == 종료')
        break
    
    after = r_json['result_data']['paging']['next_params']['after']
    after_dict[call_count*20] = after
        
    # 데이터 수집
    writers = [r_json['result_data']['items'][idx]['author']['name'] for idx in range(20)]
    contents = [r_json['result_data']['items'][idx]['content'] for idx in range(20)]
    dates = [datetime.datetime.fromtimestamp(round(r_json['result_data']['items'][idx]['created_at']/1000)).strftime('%Y-%m-%d %H:%M:%S') for idx in range(20)]
    comment_counts = [r_json['result_data']['items'][idx]['comment_count'] for idx in range(20)]
    emotion_counts = [r_json['result_data']['items'][idx]['emotion_count'] for idx in range(20)]
    
    # 데이터 프레임에 추가
    tmp = pd.DataFrame({'작성자':writers,
                         '일자':dates,
                         '댓글개수':comment_counts,
                         '감정표현개수':emotion_counts,
                         '내용':contents})
    
    df = df.append(tmp, ignore_index=True)    
    
    # log
    if df.shape[0] % 100 == 0:
        print(f'{df.shape[0]}개 데이터 수집 완료 == 경과시간: {round(time.time()-t1)}초')
        time.sleep(1)
    
    # log2
    if df.shape[0] % 2000 == 0:
        print(f'{df.shape[0]}개 데이터 수집 완료 == 10초 휴식!!')
        time.sleep(10)
    
    # call_count += 1
    call_count += 1 

# 저장

In [ ]:
df.to_excel(DATAFRAME_SAVE_PATH, index=False)
pd.DataFrame(after_dict.items(), columns = ['마지막개수','after값']).to_excel(API_FILE_SAVE_PATH, index=False)